## GROUP : BERTELLI, BABINET, HAMIDI

# TC4 - Lab exercises 2

In this notebook, we are going to improve the POS tagger of last week. Instead of using a naive Bayes classifier, we will rely on a HMM where:
- the hidden states are POS tags
- the observations are words

It is a first order HMM where probabilities are defined as follows:
$$
p(y_1...y_n, x_1...x_n) = p(y_1) \prod_{i=2}^n p(y_i | y_{i-1}) \prod_{i=1}^n p(x_i | y_i)
$$

In [1]:
import nltk
import numpy as np
import sys
import math
from collections import defaultdict
from sklearn.model_selection import train_test_split

# 1. Data

We first need to load and split the data between train and test data. You need to report the code from last week with the same split (90% train / 10% test).

In [2]:
# import dataset
data = nltk.corpus.brown.tagged_sents(tagset='universal')

In [3]:
train_data, test_data = train_test_split(data, test_size = 0.1)

In [4]:
len(train_data), len(test_data)

(51606, 5734)

We will store the parameters of the HMM in numpy arrays. Therefore, to simplify the model, we rely on a dictionnary that maps words to tokens:
- the constructor takes as argument an iteratable over strings (e.g. list of strings) containing the vocabulary to store in the dictionnary
- you can set unk="\*UNK\*" to add entry for unknown strings (do not do it for POS tags!)
- len(dict) gives you the numbers of entry in the dict
- str_to_id maps a string to an index
- id_to_str gives you the string stored at a given index

In [5]:
class Dict:
    def __init__(self, words, unk=None):
        self._unk = unk
        self._word_to_id = dict()
        self._id_to_word = list()

        if unk in words:
            raise RuntimeError("UNK word exists in vocabulary")

        if unk is not None:
            self.unk_index = self._add_word(unk)

        for word in words:
            self._add_word(word)

    # for internal use only!
    def _add_word(self, word):
        if word not in self._word_to_id:
            id = len(self._id_to_word)
            self._word_to_id[word] = id
            self._id_to_word.append(word)
            return id
        else:
            return self._word_to_id[word]

    def str_to_id(self, word):
        if self._unk is not None:
            return self._word_to_id.get(word, self.unk_index)
        else:
            return self._word_to_id[word]

    def id_to_str(self, id):
        return self._id_to_word[id]

    def __len__(self):
        return len(self._word_to_id)

    def has_unk(self):
        return self._unk is not None
    
    def unk(self):
        return self.unk_index

Example:

In [6]:
test_dict = Dict(["a", "b", "c"], unk="*UNK*")
print("N. entry: ", len(test_dict))
print("Index of \"b\":", test_dict.str_to_id("a"))
# the following line does not throw an error because we gave a unk word
print("Index of \"e\":", test_dict.str_to_id("e"))

N. entry:  4
Index of "b": 1
Index of "e": 0


We now build the dictionnary of words and tags. We will restrict the dictionnary of words to contain only words that appears 10 or more times in the training data (use the code of last time).

For the dictionnary of words, set the unk parameters to any string you want. For the dictionnary of POS tags, do not set an unk word!

In [7]:
distribution_per_word = {}
distribution_per_word_correct = {}


for s in train_data:
    for w, tag in s:
        if w in distribution_per_word : 
            if tag in distribution_per_word[w]:
                distribution_per_word[w][tag]+=1
            else:
                distribution_per_word[w][tag] = 1
        else:
            distribution_per_word[w]={}
            distribution_per_word[w][tag] = 1

for w in distribution_per_word:
    total = sum(distribution_per_word[w].values())
    if (total>=10): 
        distribution_per_word_correct[w] = distribution_per_word[w]
        

In [8]:
tmp = [list(x.keys()) for x in distribution_per_word_correct.values()]

tags = [item for sublist in tmp for item in sublist]

word_dict = Dict(distribution_per_word_correct.keys(), unk = "**UNK**")
tags_dict = Dict(tags)


# 2. Hidden Markov Model

The HMM class is a simple container for:
- the dictionnary of hidden states y_dict (i.e. dictionnary of tags)
- the dictionnary of observations x_dict (i.e. dictionnary of words)
- the parameters of the HMM:
    * init_prob $\in \mathbb R^{|Y|}$: initial tag probabilities $p(y_0) = init\_prob[y_0]$
    * transition_prob $\in \mathbb R^{|Y| \times |Y|}$: tag transition probabilities $p(y_i | y_{i - 1}) = transition\_prob[y_{i - 1}, y_i]$
    * observation_prob $\in \mathbb R^{|Y| \times |X|}$: observation probabilities $p(x_i | y_i) = observation\_prob[y_i, x_i]$

In [9]:
class HMM:
    def __init__(self, y_dict, x_dict):
        if not isinstance(y_dict, Dict) or not isinstance(x_dict, Dict):
            raise RuntimeError("Arguments must be of type Dict")

        self.y_dict = y_dict
        self.x_dict = x_dict

        n_y = len(y_dict)
        n_x = len(x_dict)
        self.init_prob = np.zeros((n_y,), float) 
        self.transition_prob = np.zeros((n_y, n_y), float) 
        self.observation_prob = np.zeros((n_y, n_x), float) 

## 2.1 Learning

Compute the matrices of probabilities hmm.init_prob, hmm.observation_prob and hmm.transition_prob from the data.

You **must** smooth the distributions!

In [10]:
hmm = HMM(tags_dict, word_dict)

###init prob###
#computing the frequency of each tag to be the first tag of the sequence
for sent in train_data:
    tag = sent[0][1]
    id_tag = tags_dict.str_to_id(tag)
    hmm.init_prob[id_tag]+=1
#smooting
hmm.init_prob+=1
hmm.init_prob/=(len(tags_dict)+len(train_data))


###transition prob###
#computing p(yi|y(i-1)) 
d_tag = defaultdict(int)
for sent in train_data:
    for i in range(1,len(sent)):
        cur_tag = sent[i][1]
        pred_tag = sent[i-1][1]
        id_cur_tag = tags_dict.str_to_id(cur_tag)
        id_pred_tag = tags_dict.str_to_id(pred_tag)
        hmm.transition_prob[id_pred_tag][id_cur_tag]+=1
        d_tag[id_pred_tag]+=1
#smoothing
hmm.transition_prob+=1
for id_tag in d_tag:
    hmm.transition_prob[id_tag,:]/=(d_tag[id_tag]+len(tags_dict))   
    
###observation prob###
d_tag = defaultdict(int)
for sent in train_data:
    for i in range(len(sent)):
        cur_tag = sent[i][1]
        cur_w = sent[i][0]
        id_cur_tag = tags_dict.str_to_id(cur_tag)
        id_cur_w = word_dict.str_to_id(cur_w)
        hmm.observation_prob[id_cur_tag][id_cur_w]+=1
        d_tag[id_cur_tag]+=1
#smoothing
hmm.observation_prob+=1
for id_tag in d_tag:
    hmm.observation_prob[id_tag,:]/=(d_tag[id_tag]+len(word_dict))   

The following three cells check that the distribution you computed correctly sum to one. The first cell should output 1.0, the two others should output array containing twelve times the number 1.

In [11]:
hmm.init_prob.sum()

0.9999999999999999

In [12]:
hmm.transition_prob.sum(1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [13]:
hmm.observation_prob.sum(1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

# 2.2. Viterbi

Implement the viterbi **without** computing in the log domain. What tagging accuracy do you achieve? How is it compared to the naive Bayes model of last week?

In [14]:
def viterbi(hmm, words):
    """
    Input:
    - hmm: an HMM object
    - words: a list of words (ie a sentence)
    Return:
    - a list of POS tags
    """

    #DEFINING THE CHART AND BACKPOINTER TABLES
    chart = np.zeros((len(hmm.y_dict), len(words)), float)
    backpointer = np.zeros((len(hmm.y_dict), len(words)), float)
    
    #FILLING THE FIRST LINE OF THE CHART TABLE 
    for i in range(len(hmm.y_dict)):
        id_w0 = word_dict.str_to_id(words[0])
        chart[i,0] = hmm.init_prob[i]*hmm.observation_prob[i,id_w0]
        
    #FILLING BACKPOINTER TABLE AND THE REST OF THE CHART TABLE
    #for each word
    for i in range(1,len(words)):
        id_w = word_dict.str_to_id(words[i])
        #for each possible tag 
        for j in range(len(hmm.y_dict)):
            b_score = -1.0
            #for each possible (tag, tag') we want the maximum of the equation below
            for k in range(len(hmm.y_dict)):
                score = hmm.transition_prob[k,j]*hmm.observation_prob[j,id_w]*chart[k,i-1]
                #if the score is superior, we update because it wasn't the maximun
                if(score>b_score):
                    chart[j,i] = score
                    b_score = score
                    backpointer[j,i] = k
    
    #FILLING THE TABLE CONTAINING THE ID OF THE GOOD TAGS
    y = np.zeros(len(words))
    y[len(words)-1] = np.argmax(chart[:,len(words)-1], axis=0)

    for j in range(1,len(words))[::-1]:
        y[j-1] = backpointer[int(y[j-2]),j]
    #MAPPING EACH ID TAG TO THE TAG
    pred = [tags_dict.id_to_str(int(i)) for i in (y)]

    return pred

In [15]:
tags_dict.id_to_str(0)

'PRON'

In [16]:
viterbi(hmm,['the', 'cat', 'is', 'black'])

['DET', 'NOUN', 'VERB', 'ADJ']

In [17]:
# Evaluate the HMM using the viterbi
n_tags = 0
n_correct_tags = 0
for sentence in test_data:
    words = [w for w, t in sentence]
    pred = viterbi(hmm, words)
    n_tags += len(sentence)
    n_correct_tags += sum(1 for w in range(len(sentence))  if sentence[w][1] == pred[w])

print("Tagging accuract: %.2f" % (100 * n_correct_tags / n_tags))

Tagging accuract: 88.79


## ANSWER : 
Last week, with the naive bayes implementation, we got a maximun of 86% of accuracy, and now we get almost 89% of accuracy.
So the viterbi and hmm model performs better than the naie Bayes model. This is quite logical, because last  week our model didn't take into account the words before, but just the word to predict the tag. Now our model takes into account the tag that precede the current words, and so performs better.

# 2.3. Viterbi in the log domain

Copy/paste you code from the previous cell and change it to compute in the log domain. What tagging accuracy do you achieve? How is it compared to the naive Bayes model of last week and to the previous implementation of the viterbi?

In [18]:
def viterbi_log(hmm, words):
    """
    Input:
    - hmm: an HMM object
    - words: a list of words (ie a sentence)
    Return:
    - a list of POS tags
    """
    
    
    chart = np.zeros((len(hmm.y_dict), len(words)), float)
    backpointer = np.zeros((len(hmm.y_dict), len(words)), float)
    for i in range(len(hmm.y_dict)):
        id_w0 = word_dict.str_to_id(words[0])
        chart[i,0] = np.log(hmm.init_prob[i]) +np.log(hmm.observation_prob[i,id_w0])
        
    for i in range(1,len(words)):
        id_w = word_dict.str_to_id(words[i])
        for j in range(len(hmm.y_dict)):
            b_score = -float('Inf')
            for k in range(len(hmm.y_dict)):
                score = np.log(hmm.transition_prob[k,j]) + np.log(hmm.observation_prob[j,id_w])+ (chart[k,i-1])
                if(score>b_score):
                    chart[j,i] = score
                    b_score = score
                    backpointer[j,i] = k
    #print(chart)           
    y = np.zeros(len(words))
    y[len(words)-1] = np.argmax(np.exp(chart[:,len(words)-1]), axis=0)

    for j in range(1,len(words))[::-1]:
        y[j-1] = backpointer[int(y[j-2]),j]
    pred = [tags_dict.id_to_str(int(i)) for i in (y)]

    return pred

In [19]:
# Evaluate the HMM using the viterbi in the log domain

n_tags = 0
n_correct_tags = 0
i =0
for sentence in test_data:
    words = [w for w, t in sentence]
    pred = viterbi_log(hmm, words)
    n_tags += len(sentence)
    n_correct_tags += sum(1 for w in range(len(sentence))  if sentence[w][1] == pred[w])

print("Tagging accuract: %.2f" % (100 * n_correct_tags / n_tags))

Tagging accuract: 88.79


### ANSWER: 
As before, for the "normal" viterbi, we get better results than the naive bayes of last week, and this for the same reason.
We get the same result. Indeed, the log viterbi is usefull if we have paths (ie sequences of tags), that are large and so the resulting probabilities, which are the product, are almost equal to zero. Here, as the results are the same, we deduce, that the paths aren't described by a product which is too small.

However, we should keep the log probabilities because it doesn't affect  badly the performance of our model, and if these products mentionned before tend to zero, it can make it more efficient.

# 3. Marginalization

As a last exercise, implement function that evaluate the probability of a sequence of words and a sequence of hidden states given a HMM.

In [20]:
def probabilit_y(hmm, tags):
    p = hmm.init_prob[tags_dict.str_to_id(tags[0])]
    for i in range (1, len(tags)):
        cur_id_tag = tags_dict.str_to_id(tags[i])
        pred_id_tag = tags_dict.str_to_id(tags[i-1])
        p *=  hmm.transition_prob[pred_id_tag][cur_id_tag]
    return(p)  

In [21]:
import random
tags = "DET NOUN VERB DET ADJ NOUN .".split()
print(probabilit_y(hmm, tags))
random.shuffle(tags)
print(probabilit_y(hmm, tags))

0.00015516963615024506
1.7184983167278761e-06


### ANSWER:

So we can see that the initial structure which is quite a normal one, has a probability of 10^-4, and the other one, obtained after a random shuffle, has a probability of 10^-6.
Thus, this is logical because the first structure is normal and the second one random. We can judge wether our structure, is logical or not. The higher the probability is the most likely the sequence is possible.

Thus, it allows us to judge after how "sure" we are about the sequence of tags emitted.

In [22]:
def probabilit_x(hmm, words):
    chart = np.zeros((len(hmm.y_dict), len(words)), float)
    for i in range (len(hmm.y_dict)):
        chart[i,0] = hmm.init_prob[i] * hmm.observation_prob[i][word_dict.str_to_id(words[0])]

    for j in range(1,len(words)):
        for i in range(len(hmm.y_dict)):
            for k in range(len(hmm.y_dict)):
                chart[i,j]+= hmm.transition_prob[k][i]*hmm.observation_prob[k][word_dict.str_to_id(words[j])] * chart[k, j-1]

    return(np.sum(chart[:,len(words)-1]))

In [23]:
sentence = "This is a sentence .".split()
print(probabilit_x(hmm, sentence))
random.shuffle(sentence)
print(probabilit_x(hmm, sentence))

2.0913331285286098e-15
3.6403486468072695e-17


###  ANSWER :

In this case we observe that the probability of the real sentence is higher (100 x more) than the random sentence. Thus this confirms us our HMM model works